In [ ]:
# 필요한 module import

import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Data Loading

k_train_mnist = pd.read_csv("./data/digit-recognizer/train.csv")
k_test_mnist = pd.read_csv("./data/digit-recognizer/test.csv")

tf.reset_default_graph()

scaler = MinMaxScaler()
scaler.fit(k_train_mnist)

x_data = scaler.fit_transform(k_train_mnist.iloc[:,1:])
y_data = pd.get_dummies(k_train_mnist["label"]).values

pre_x_data = scaler.fit_transform(k_test_mnist.values)


# train / test 구분
train_num = int(x_data.shape[0] * 0.8)

# train data set
train_x_data = x_data[:train_num]
train_y_data = y_data[:train_num]

# test data set
test_x_data = x_data[train_num:]
test_y_data = y_data[train_num:]

# 함수로 모델 1개 만드는 로직 짜기


result = []
def make_H(X,Y,d_rate):
    tf.reset_default_graph()
    X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    d_rate = tf.placeholder(dtype=tf.float32)
    
    x_img = tf.reshape(X, [-1,28,28,1])
    L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                             kernel_size=[3,3], strides=1,
                             padding="SAME", activation=tf.nn.relu)
    L1 = tf.layers.max_pooling2d(inputs=L1, pool_size = [2,2],
                                   padding="SAME", strides=2)
    L1 = tf.layers.dropout(inputs=L1, rate=d_rate)
    
    L2 = tf.layers.conv2d(inputs=L1, filters=64,
                             kernel_size=[3,3], strides=1,
                             padding="SAME", activation=tf.nn.relu)
    L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],
                                    padding="SAME", strides=2)
    L2 = tf.layers.dropout(inputs=L2, rate=d_rate)
    
    Dl3 = tf.reshape(L2, [-1, 7*7*64])
    
    Dl3 = tf.layers.dense(inputs=Dl3, units=256, activation=tf.nn.relu)
    
    Dl4 = tf.layers.dense(inputs=Dl3, units=128, activation=tf.nn.relu)
    
    Dl5 = tf.layers.dense(inputs=Dl4, units=512, activation=tf.nn.relu)
    
    H = tf.layers.dense(inputs=Dl5, units=10)
    
    cost = tf.losses.softmax_cross_entropy(Y, H) # 순서가 바뀐다

    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    train_epoch = 30
    batch_size = 100

    for step in range(train_epoch):
        num_of_iter = int(train_x_data.shape[0]/batch_size)
        cost_val = 0
        for i in range(num_of_iter):
            start = i * batch_size
            end = start + batch_size
            cut_train_x = train_x_data[start:end]
            cut_train_y = train_y_data[start:end]
            _, cost_val = sess.run([train, cost], 
                                   feed_dict={ X : cut_train_x,
                                                Y : cut_train_y,
                                                 d_rate : 0.8})

        if step % 3 == 0:
            print("Cost값 : {}".format(cost_val))

     
        result.append(list(sess.run(H, feed_dict={X : test_x_data}))[0])
        result_H = result
            
    return result_H


for _ in range(2):
    a=make_H(train_x_data, train_y_data, 0.2)
    print(a)

